In [1]:
import warnings
import pylj
from pylj import comp, mc, sample, util
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
def mc_simulation(number_of_particles, temperature, box_length, number_of_steps, sample_frequency):
    # Creates the visualisation environment
    %matplotlib notebook
    # Initialise the system placing the particles on a square lattice
    system = mc.initialise(number_of_particles, temperature, box_length, 'square')
    # This sets the sampling class as Energy, which shows the energy of the system
    sample_system = sample.Energy(system)
    # Compute the energy of the system
    system.particles, system.distances, system.energies = comp.compute_energy(system.particles, 
                                                                              system.box_length,
                                                                              system.cut_off)
    old_energy = system.energies.sum()
    # Add this energy to the energy sample array
    system = mc.sample(old_energy, system)
    # Begin the monte carlo loop
    for i in range(0, number_of_steps):
        system.step += 1
        # Select a random particle to remove
        random_particle, system.position_store = mc.select_random_particle(system.particles)
        # Select a random position to replace that particle
        system.particles = mc.get_new_particle(system.particles, random_particle, system.box_length)
        # Compute the new energy of the system
        system.particles, system.distances, system.energies = comp.compute_energy(system.particles, 
                                                                                  system.box_length, 
                                                                                  system.cut_off)
        new_energy = system.energies.sum()
        # Assess the Metropolis condition
        if mc.metropolis(temperature, old_energy, new_energy):
            old_energy = mc.accept(new_energy)
        else:
            system.particles = mc.reject(system.position_store, system.particles, random_particle)
        # Add this energy to the energy sample array
        system = mc.sample(old_energy, system)
        # At a given frequency sample the positions and plot
        if system.step % sample_frequency == 0:
            sample_system.update(system)
    return system

The `mc_simulation` function takes five variables:
- The number of particles
- The simulation temperature (for the Metropolis condition)
- The simulation cell vector
- The number of steps
- The sampling frequency (how often the image is updated)

In [ ]:
system = mc_simulation(75, 273.15, 45, 100000, 100)